# Best practices

1. _Experienced_ Django developers _stay away_ from trying to change settings in **production** since they require 
**server restart**.
2.  All settings files need to be version control >
3. Don't Repeat youself(DRY) > you should inherit from a base settings file rather than cutting and pasting from one from one file to another.
4. Keep secret key safe > they should be kept out of version control

5. SECRET_KEY, _production database passwords_ and API key settings to services like Amazon, stripe.AWS Keys , oauth_tokens also need to be protected.

### Using Multiple Settings Files
1. you will have settings directory containing your settings files.something like follwoing


settings/

    __init__.py
    base.py
    local.py
    staging.py
    test.py
    production.py

base.py - settings common to all instances of the project.

local.py - when you are working on project locally.settings include **Debug mode**, log_level, activation of developer tools like django-bebug-toolbar dev name it dev.py.

staging.py - semi-private version of site on production server.Managers and clients would be looking at it before your work is moved to production.

test.py - settings for running tests including test runners, in-memory database definitions and log settings.

production.py - servers that hosts the real websites, sometimes called as prod.py.

2. if you have special-purpose servers, you might have custom settings files for each of them. 

python manage.py shell --settings=settings.local_siddhant

python manage.py runserver --settings=settings.local_siddhant 8000

python manage.py runserver --settings=settings.local

python manage.py shell --settings=settings.staging

python manage.py runserver --settings=settings.production


3. django-admin.py at the command line works without the --settings option.

### A development settings Example

In [ ]:
# settings/local.py
from .base import *

DEBUG = True
TEMPLATE_DEBUG = DEBUG

EMAIL_BACKEND = 'django.core.mail.backends.console.EmailBackend'

DATABASES = {
    "default": {
        "ENGINE": "django.db.backends.postgresql_psycopg2",
        "NAME": "twoscoops",
        "USER": "postgres",
        "PASSWORD": "",
        "HOST": "localhost",
        "PORT": "",
        
    }
}

INSTALLED_APPS += ("debug_toolbar", )

### Multiple Development Settings 

In [ ]:
# settings/dev_siddhant.py

from .local import *

# set short cache timeout 
CACHE_TIMEOUT = 30


setting/

    __init__.py
    base.py
    dev_siddhant.py
    dev_jassim.py
    local.py
    staging.py
    test.py
    production.py
    

### Separate Configuration from code

1. Secret keys are configuration values, not code.
2. They should be kept secret. Keeping them in version control means that everyone with respository access has access to them.
3. PAAS(platform as a service) don't give ability to edit code on various severs.Even if they allow it it's  a terribly -dangerous- pratice.

4. Most PAAS recommend the use of environment variables for configuration and have builtin features for setting and managing them.


In [ ]:
$ export SOME_SECRET_KEY=Lc3-cr43am-15-yummy
$ export AUDREY_FREEZER_KEY=Y34h-d0nt=t0ouch-my-lc3-cr34m

### How to set Environment Variables in Production

In [ ]:
$ heroku config:set SOME_SECRET_KEY=lc3-cr3am-15-yummy

In [ ]:
>>> import os
>>> os.enivron["Some_SECRET_KEY"]
"1c3-cr3am-15-yummy"

In [ ]:
# Top of settings/production.py 
import os
SOME_SECRET_KEY = os.environ["SOME_SECRET_KEY"]


snippet simply gets the value of the SOME_SECRET_KEY environmental variable from the operating system and saves it to a python variable called SOME_SECRET_KEY.


### Handling missing secret key exceptions

In [ ]:
# settings/base.py
import os 

# Normally you should not import ANYTHING from django directly into your settings
#but ImproperlyConfigured is an exception 
from django.core.exceptions import Improperly Configured

def get_any_variable(var_name):
    """Get the environment variable or return exception"""
    try:
        return os.environ[var_name]
    except KeyError:
        error_msg = "Set the {} environment variable".format(var_name)
        raise ImproperlyConfigured(error_msg)

In [ ]:
SOME_SECRET_KEY = get_env_variable("SOME_SECRET_KEY")

### Secret files pattern

In [ ]:
# create a file using conf. format of choice JSON YAML XML
# Add a secrets loader 
# Add the secrets file name to the .gitignore or .hgignore
# json file -
{
    "Filename": "",
    "SECRET_KEY": "I've got a secret:",
    "DATABASES_HOST": "127.0.0.1",
    "PORT": "5432",
}

# settings/base.py
import json
from django.core.exceptions import ImproperlyConfigured

with open("secrets,json") as f:
    secrets = f.loads(f.read())
    
def get_secret(setting, secrete=secrets):
    try:
        return(secrets[setting])
    except KeyError:
        error_msg = "Set the {} environmental variable".format(setting)
        raise ImproperlyConfigured(error_msg)
        
SECRET_KEY = get_secret("SECRET_KEY")
        

### Using Multiple Requirements Files

requirements/
    
    base.txt
    local.txt
    staging.txt
    production.txt

In the _base.txt_ file place the dependencies used in all environments
Example 


 Django==1.6.0
 
 psycopg2==2.5.1
 
 South==0.8.4
 
Your _local.txt_ file should have dependencies used for local developments auch as :
Example
 
 -r base.txt #includes the base.txt requirements file
 
 coverage==3.7
 
 django-debug-toolbar==0.11
 
### Installing from Multiple Requirements files

In [ ]:
$ pip install -r requirements/local.txt
$ pip install -r requirements/production.txt


### Handling File Paths in Settings
Using Unipath

In [ ]:
# at the top of settings/base.py
from unipath import Path

BASE_DIR = Path(__file__).ancestor(3)
MEDIA_ROOT = BASE_DIR.child("media")
STATIC_ROOT = BASE_DIR.child("static")
STATICFILES_DIRS = (
    BASE_DIR.child("assets"),
)
TEMPLATE_DIRS = (
    BASE_DIR.child("templates"),
)
